In [1]:
#spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1690140558302_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import json
from datetime import timedelta, datetime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#Importação da biblioteca PySpark para processamento de dados em larga escala
from pyspark.sql import SparkSession, Row
from decimal import Decimal
from pyspark.sql.functions import lit, col, expr, to_timestamp, format_number,udf, when
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, TimestampType, DateType, ArrayType, DecimalType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.appName('Nubank').getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## DataFrames

#### Montando a tabela de Transacoes

In [5]:
#transformando json em dataframe
compras = spark.read.json('s3://nubank-api-dependencias/JSONS/compras.json')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Converte a coluna 'time' para o formato Timestamp e divide a coluna 'amount' por 100
compras = compras.withColumn('time', to_timestamp(compras['time'], 'yyyy-MM-dd\'T\'HH:mm:ssX'))
compras = compras.withColumn('amount', col("amount") / 100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Adicionar uma coluna vazia ao DataFrame df_transacoes para corresponder ao DataFrame df_parcelas
compras = compras.withColumn("qtd_parcelas", lit(None).cast("integer"))
compras = compras.withColumn("parcela", lit(None).cast("integer"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#Removendo colunas desnecessarios
compras = compras.drop(*['_links','account','amount_without_iof','details','href','id','source','tokenized'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
schema_ = StructType([
    StructField('amount', DoubleType(), True),
    StructField('category', StringType(), True),
    StructField('description', StringType(), True),
    StructField('time', TimestampType(), True),
    StructField('title', StringType(), True),
    StructField('qtd_parcelas', IntegerType(), True),
    StructField('parcela', IntegerType(), True)
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
parcelas = spark.read.json('s3://nubank-api-dependencias/JSONS/parcelas.json',schema_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Converte a coluna 'time' para o formato Timestamp
parcelas = parcelas.withColumn('time', to_timestamp(parcelas['time'], 'yyyy-MM-dd\'T\'HH:mm:ssX'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
parcelas = parcelas.withColumn("parcela", col("parcela").cast("integer"))
parcelas = parcelas.withColumn("qtd_parcelas", col("qtd_parcelas").cast("integer"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
transacoes = compras.union(parcelas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Substituir os nulls por 0 na coluna "qtd_parcelas"
transacoes = transacoes.fillna(0, subset=["qtd_parcelas"])

# Substituir os nulls por 0 na coluna "parcela"
transacoes = transacoes.fillna(0, subset=["parcela"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Convertendo o DataFrame em parquet
transacoes.write.\
parquet('s3://nubank-api-dependencias/Tabelas/transacoes/',mode='overwrite')

#### Limite

In [ ]:
limite = spark.read.json('s3://nubank-api-dependencias/JSONS/limite.json')

In [ ]:
limite = limite.withColumn('Data', to_timestamp(limite['Data'], 'yyyy:MM:dd HH:mm:ss'))

In [ ]:
#Nome parquet
limite_name = datetime.now().strftime('%Y:%m:%d %H:%M:%S')
limite_parquet_name = 'limite-'+limite_name

In [ ]:
#Convertendo o DataFrame em parquet
limite.write.parquet(f's3://nubank-api-dependencias/Tabelas/limite/{limite_parquet_name}.parquet')

#### Saldo

In [ ]:
saldo = spark.read.json('s3://nubank-api-dependencias/JSONS/saldo.json')

In [ ]:
saldo = saldo.withColumn('Data', to_timestamp(saldo['Data'], 'yyyy:MM:dd HH:mm:ss'))

In [ ]:
#Nome parquet
saldo_name = datetime.now().strftime('%Y:%m:%d %H:%M:%S')
saldo_parquet_name = 'saldo-'+saldo_name

In [ ]:
#Convertendo o DataFrame em parquet
saldo.write.parquet(f's3://nubank-api-dependencias/Tabelas/saldo/{saldo_parquet_name}.parquet')

#### Despesas

In [ ]:
schemas = StructType([
    StructField('postDate', StringType(), True),
    StructField('title', StringType(), True),
    StructField('amount', FloatType(), True)
])

In [ ]:
despesas = spark.read.json('s3://nubank-api-dependencias/JSONS/despesas.json',schemas)

In [ ]:
despesas = despesas.withColumn('postDate', to_timestamp(despesas['postDate'], 'yyyy-MM-dd'))

In [ ]:
#Convertendo o DataFrame em parquet
despesas.write.\
parquet('s3://nubank-api-dependencias/Tabelas/despesas/', mode='overwrite')

#### Receitas

In [ ]:
receitas = spark.read.json('s3://nubank-api-dependencias/JSONS/receitas.json',schemas)

In [ ]:
receitas = receitas.withColumn('postDate', to_timestamp(receitas['postDate'], 'yyyy-MM-dd'))

In [ ]:
#Convertendo o DataFrame em parquet
receitas.write.\
parquet('s3://nubank-api-dependencias/Tabelas/receitas/', mode='overwrite')